# am'

In [ ]:
//// test

open testing

In [ ]:
open rust_operators

## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"[$0]\")>]\n#endif\ntype Slice<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"_\")>]\n#endif\ntype Slice'<'T> = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"Vec<$0>\")>]\n#endif\ntype Vec<'T> = class end"

## length

In [ ]:
inl length forall dim {int} el. (a : a dim el) : dim =
    a |> length

## index

In [ ]:
inl index forall dim {int} el. (i : dim) (a : a dim el) : el =
    index a i

## base

In [ ]:
inl base forall dim {int} el. ((a a) : a dim el) : array_base el =
    a

## base'

In [ ]:
inl base' forall el. ((a a) : a int el) : array_base el =
    a

## generic_equable

In [ ]:
inl generic_equable x1 x2 =
    if length x1 <>.. length x2
    then false
    else
        let rec loop i =
            if i < length x1
            then index i x1 = index i x2 && loop (i + 1)
            else true
        loop 0

## equable

In [ ]:
instance equable a dim {number; int} t = generic_equable

## append

In [ ]:
instance append a dim {int; number} t = am.append

In [ ]:
//// test
///! fsharp
///! rust
///! typescript
///! python

a' ;[ -1i64; 0 ] ++ a' ;[ 1; 2 ]
|> _assert_eq (a' ;[ -1; 0; 1; 2 ])

.rs output:
assert_eq / actual: Array(MutCell([-1, 0, 1, 2])) / expected: Array(MutCell([-1, 0, 1, 2]))

.ts output:
assert_eq / actual: -1,0,1,2 / expected: -1,0,1,2

.py output:
assert_eq / actual: [-1, 0, 1, 2] / expected: array('q', [-1, 0, 1, 2])


.fsx output:
assert_eq / actual: [|-1L; 0L; 1L; 2L|] / expected: [|-1L; 0L; 1L; 2L|]


## collect

In [ ]:
inl collect forall t r. (fn : t -> a int r) (items : a int t) : a int r =
    items
    |> am.map fn
    |> am.fold (++) (a ;[])

## init

In [ ]:
inl init n : a int _ =
    am.init n id

## choose

In [ ]:
inl choose f l =
    (l, [])
    ||> am.foldBack fun x acc =>
        match f x with
        | Some y => y :: acc
        | None => acc
    |> listm.toArray

In [ ]:
//// test
///! fsharp
///! typescript
///! python

10
|> init
|> choose (fun x => if x % 2 = 0 then Some x else None)
|> _assert_eq (a' ;[ 0; 2; 4; 6; 8 ])

.ts output:
assert_eq / actual: 0,2,4,6,8 / expected: 0,2,4,6,8

.py output:
assert_eq / actual: [0, 2, 4, 6, 8] / expected: array('l', [0, 2, 4, 6, 8])


.fsx output:
assert_eq / actual: [|0; 2; 4; 6; 8|] / expected: [|0; 2; 4; 6; 8|]


## sum

In [ ]:
inl sum a =
    a |> am.fold (+) 0

In [ ]:
//// test
///! fsharp
///! rust
///! typescript
///! python

10
|> init
|> sum
|> _assert_eq 45

.rs output:
assert_eq / actual: 45 / expected: 45

.ts output:
assert_eq / actual: 45 / expected: 45

.py output:
assert_eq / actual: 45 / expected: 45


.fsx output:
assert_eq / actual: 45 / expected: 45


## init_series

In [ ]:
inl init_series start end inc : a int _ =
    inl total = conv ((end - start) / inc) + 1
    am.init total (conv >> (*) inc >> (+) start)

In [ ]:
//// test
///! fsharp
///! rust
///! typescript
///! python

init_series 0 1 0.5
|> _assert_eq (a ;[ 0f64; 0.5; 1 ])

.rs output:
assert_eq / actual: Array(MutCell([0.0, 0.5, 1.0])) / expected: Array(MutCell([0.0, 0.5, 1.0]))

.ts output:
assert_eq / actual: 0,0.5,1 / expected: 0,0.5,1

.py output:
assert_eq / actual: [0.0, 0.5, 1.0] / expected: array('d', [0.0, 0.5, 1.0])


.fsx output:
assert_eq / actual: [|0.0; 0.5; 1.0|] / expected: [|0.0; 0.5; 1.0|]


## head

In [ ]:
inl head (ar : a _ _) =
    if var_is ar || length ar > 0
    then ar |> index 0
    else error_type "The length of the array should be greater than 0."

## last

In [ ]:
inl last (ar : a _ _) =
    inl len = length ar
    if var_is ar || len > 0
    then ar |> index (len - 1)
    else error_type "The length of the array should be greater than 0."

In [ ]:
//// test
///! fsharp
///! rust
///! typescript
///! python

10
|> init
|> last
|> _assert_eq 9

.rs output:
assert_eq / actual: 9 / expected: 9

.ts output:
assert_eq / actual: 9 / expected: 9

.py output:
assert_eq / actual: 9 / expected: 9


.fsx output:
assert_eq / actual: 9 / expected: 9


## try_pick

In [ ]:
inl try_pick forall t u. (fn : t -> option u) (array : a _ t) : option u =
    (array, None)
    ||> am.foldBack fun x acc =>
        match acc with
        | Some _ => acc
        | None => x |> fn

In [ ]:
//// test
///! fsharp
///! typescript
///! python

10
|> init
|> try_pick (fun x => if x = 5i32 then Some x else None)
|> _assert_eq (Some 5i32)

.ts output:
assert_eq / actual: US0_0 5 / expected: US0_0 5

.py output:
assert_eq / actual: US0_0 5 / expected: US0_0 5


.fsx output:
assert_eq / actual: US0_0 5 / expected: US0_0 5


## indexed'

In [ ]:
inl indexed' forall t dim {int; number} dim' {int; number} u {number}. (ar : a dim t) : a dim' (u * t) =
    ((0, a ;[]), ar)
    ||> am.fold fun (i, acc) x =>
        i + 1, acc ++ a ;[i, x]
    |> snd

In [ ]:
//// test

am.init 3i32 ((*) 2)
|> indexed'
|> _assert_eq (a ;[0i32, 0; 1, 2; 2, 4] : a i32 _)

assert_eq / actual: [|struct (0, 0); struct (1, 2); struct (2, 4)|] / expected: [|struct (0, 0); struct (1, 2); struct (2, 4)|]


## map_base

In [ ]:
inl map_base forall t u. (fn : t -> u) (x : array_base t) : array_base u =
    a x
    |> am.map fn
    |> fun (a x : _ int _) => x

## average

In [ ]:
inl average forall el {number}. (a : a _ el) : el =
    a |> $'Array.average'

## distinct

In [ ]:
inl distinct forall dim el. (a : a dim el) : a dim el =
    a |> $'Array.distinct'

## skip

In [ ]:
inl skip forall dim el. (n : dim) (a : a dim el) : a dim el =
    a |> $'Array.skip' n

## skip_while

In [ ]:
inl skip_while forall dim el. (fn : el -> bool) (a : a dim el) : a dim el =
    a |> $'Array.skipWhile' fn

## to_list'

In [ ]:
inl to_list' forall dim t. (items : a dim t) : listm'.list' t =
    items |> $'Array.toList'

In [ ]:
//// test

a' ;[ "a"; "b" ]
|> to_list'
|> listm'.unbox
|> _assert_eq [ "a"; "b" ]

assert_eq / actual: UH0_1 ("a", UH0_1 ("b", UH0_0)) / expected: UH0_1 ("a", UH0_1 ("b", UH0_0))


## parallel_map

In [ ]:
inl parallel_map forall dim el el'. (fn : el -> el') (a : a dim el) : a dim el' =
    a |> $'Array.Parallel.map' fn

## map'

In [ ]:
inl map' forall dim el el'. (fn : el -> el') (a : a dim el) : a dim el' =
    $'!a |> Array.map !fn '

## sort_by

In [ ]:
inl sort_by forall dim el. (fn : el -> _) (a : a dim el) : a dim el =
    a |> $'Array.sortBy' fn

## sort

In [ ]:
inl sort forall dim el. (a : a dim el) : a dim el =
    a |> $'Array.sort'

## sort_descending

In [ ]:
inl sort_descending forall dim el. (a : a dim el) : a dim el =
    a |> $'Array.sortDescending'

## transpose

In [ ]:
inl transpose forall dim el. (a : a dim (a dim el)) : a dim (a dim el) =
    a |> $'Array.transpose'

## try_item

In [ ]:
inl try_item forall dim el. (i : i32) (a : a dim el) : option el =
    a |> $'Array.tryItem' i |> optionm'.unbox

## indexed

In [ ]:
inl indexed (array : a _ _) =
    (([], 0), array)
    ||> am.fold fun (acc, i) x =>
        (i, x) :: acc, i + 1
    |> fst
    |> listm.rev
    |> listm.toArray

In [ ]:
//// test

am.init 3i32 ((*) 2)
|> indexed
|> _assert_eq (a ;[0i32, 0; 1, 2; 2, 4] : a i32 _)

assert_eq / actual: [|struct (0, 0); struct (1, 2); struct (2, 4)|] / expected: [|struct (0, 0); struct (1, 2); struct (2, 4)|]


## vec

In [ ]:
nominal vec t = $'Vec<`t>'

## from_vec

In [ ]:
inl from_vec forall dim el. (vec : vec el) : a dim el =
    !\\(vec, $'"fable_library_rust::NativeArray_::array_from($0)"')

## to_vec

In [ ]:
inl to_vec forall t. (ab : array_base t) : vec t =
    !\\(ab, $'"$0.to_vec()"')

## vec_push

In [ ]:
inl vec_push forall el. (el : el) (vec : vec el) : vec el =
    inl el = join el
    inl vec = join vec
    (!\($'"true; let mut !vec = !vec"') : bool) |> ignore
    // inl vec = vec |> rust.to_mut
    (!\($'"true; !vec.push(!el)"') : bool) |> ignore
    !\($'"!vec"')

## vec_reverse

In [ ]:
inl vec_reverse forall el. (vec : vec el) : vec el =
    inl vec = join vec
    (!\($'"true; let mut !vec = !vec"') : bool) |> ignore
    (!\($'"true; !vec.reverse()"') : bool) |> ignore
    !\($'"!vec"')

## vec_retain

In [ ]:
inl vec_retain forall el. (fn : el -> bool) (vec : vec el) : vec el =
    inl vec = join vec
    inl fn = join fn
    (!\($'"true; let mut !vec = !vec"') : bool) |> ignore
    // inl vec = vec |> rust.to_mut
    (!\($'"true; !vec.retain(|x| !fn(x.clone()))"') : bool) |> ignore
    !\($'"!vec"')

## vec_sort_by_key

In [ ]:
inl vec_sort_by_key forall el t. (fn : el -> t) (vec : vec el) : vec el =
    inl vec = join vec
    inl fn = join fn
    (!\($'"true; let mut !vec = !vec"') : bool) |> ignore
    // inl vec = vec |> rust.to_mut
    (!\($'"true; !vec.sort_by_key(|x| !fn(x.clone()))"') : bool) |> ignore
    !\($'"!vec"')

## vec_extend

In [ ]:
inl vec_extend forall el. (el : vec el) (vec : vec el) : vec el =
    inl el = join el
    inl vec = join vec
    (!\($'"true; let mut !vec = !vec"') : bool) |> ignore
    // inl vec = vec |> rust.to_mut
    (!\($'"true; !vec.extend(!el)"') : bool) |> ignore
    !\($'"!vec"')

## vec_collect

In [ ]:
inl vec_collect fn vec =
    ((;[] |> to_vec), (vec |> from_vec : _ i32 _))
    ||> am.fold fun acc x =>
        acc |> vec_extend (fn x)

## vec_collect_option

In [ ]:
inl vec_collect_option vec =
    ((;[] |> to_vec |> Ok), (vec |> from_vec : _ i32 _))
    ||> am.fold fun acc x =>
        x
        |> resultm.unbox
        |> fun x =>
            match acc, x |> resultm.map optionm'.unbox with
            | Ok acc, Ok (Some x) => acc |> vec_extend x |> Ok
            | _, Error error => error |> Error
            | _ => acc

## vec_collect_into

In [ ]:
inl vec_collect_into forall (c : * -> * -> *) t e.
    (x : vec (c t e))
    : c (vec t) e
    =
    !\($'"!x.into_iter().collect()"')

## vec_mapi

In [ ]:
inl vec_mapi forall dim t u. (fn : dim -> t -> u) (ar : vec t) : vec u =
    inl fn = join fn
    inl ar = join ar
    !\($'"!ar.iter().enumerate().map(|(i, x)| !fn(i.try_into().unwrap())(x.clone())).collect::<Vec<_>>()"')

## vec_map

In [ ]:
inl vec_map forall t u. (fn : t -> u) (ar : vec t) : vec u =
    (!\($'"true; let _result : Vec<_> = !ar.into_iter().map(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 }).collect::<Vec<_>>()"') : bool) |> ignore
    !\($'"_result"')

## vec_map'''

In [ ]:
inl vec_map''' forall t u. (fn : t -> u) (ar : vec t) : vec u =
    (!\($'"true; let _result : Vec<_> = !ar.into_iter().map(|x| { //"') : bool) |> ignore
    (!\\(fn !\($'"x"'), $'"true; $0 }}).collect::<Vec<_>>(); {{ //"') : bool) |> ignore
    !\($'"_result"')

## vec_map'

In [ ]:
inl vec_map' forall t u. (fn : t -> u) (ar : vec t) : vec u =
    !\\((ar, fn), $'"$0.into_iter().map(|x| $1(x.clone())).collect::<Vec<_>>()"')

## vec_fold'

In [ ]:
inl vec_fold' forall t u. (fn : u -> t -> u) (init : u) (ar : vec t) : u =
    (!\\(ar, $'"true; let _result = $0.into_iter().fold(!init, |acc, x| { //"') : bool) |> ignore
    (!\\(fn !\($'"acc"') !\($'"x"'), $'"true; $0 })"') : bool) |> ignore
    !\($'"_result"')

## vec_for_each

In [ ]:
inl vec_for_each forall t. (fn : t -> ()) (ar : vec t) : () =
    !\\((ar, fn), $'"$0.iter().for_each(|x| { $1(x.clone()); })"')

## vec_for_each'

In [ ]:
inl vec_for_each' forall t. (fn : t -> ()) (ar : vec t) : () =
    (!\($'"true; !ar.into_iter().for_each(|x| { //"') : bool) |> ignore
    inl x = fn !\($'"x"')
    (!\($'"true; !x }}); { //"') : bool) |> ignore

## vec_filter

In [ ]:
inl vec_filter forall t. (fn : t -> bool) (ar : vec t) : vec t =
    inl fn = join fn
    inl ar = join ar
    !\($'"!ar.into_iter().filter(|x| !fn(x.clone().clone())).collect::<Vec<_>>()"')

## vec_len

In [ ]:
inl vec_len forall t. (vec : vec t) : unativeint =
    !\\(vec, $'"$0.len()"')

## vec_chunks

In [ ]:
inl vec_chunks forall t. (n : i32) (vec : vec t) : vec (vec t) =
    !\\(vec, $'"$0.chunks(!n).map(|x| x.into_iter().map(|x| x.clone()).collect::<Vec<_>>()).collect::<Vec<_>>()"')

## slice

In [ ]:
nominal slice t = $'Slice<`t>'

## slice

In [ ]:
inl slice forall dim {int; number} el. from nearTo s : a dim el =
    am.slice { from nearTo } s

In [ ]:
//// test

inl x : _ i32 _ = a ;[ 1i32; 2; 3 ]
x |> slice 0 0 |> _assert_eq (a ;[])
x |> slice 0 1 |> _assert_eq (a ;[ 1 ])
x |> slice 1 1 |> _assert_eq (a ;[])
x |> slice 1 2 |> _assert_eq (a ;[ 2 ])
x |> slice 2 2 |> _assert_eq (a ;[])
x |> slice 0 2 |> _assert_eq (a ;[ 1; 2 ])

assert_eq / actual: [||] / expected: [||]
assert_eq / actual: [|1|] / expected: [|1|]
assert_eq / actual: [||] / expected: [||]
assert_eq / actual: [|2|] / expected: [|2|]
assert_eq / actual: [||] / expected: [||]
assert_eq / actual: [|1; 2|] / expected: [|1; 2|]


## range

In [ ]:
union range dim =
    | Start : dim
    | End : dim -> dim

inl range start end s =
    inl start, end =
        match start, end with
        | Start start, End fn =>
            start, s |> length |> conv |> fn
        | End start_fn, End end_fn =>
            inl len = s |> length |> conv
            start_fn len, end_fn len
    s |> slice (start |> unbox) (end |> unbox)

## slice'

In [ ]:
nominal slice' el dim = $'Slice\'<`el>'

## slice_singleton

In [ ]:
inl slice_singleton forall dim el. (x : option el) : slice' el dim =
    match x with
    | Some x => !\($'"[!x]"')
    | None =>
        !\($'"[\\\"\\\".to_string()]"') : slice' el dim
            // emit_expr `(()) `(slice' el dim) () ($'"[@dim]"' : string) : slice' el 10
            // !\( : string) : slice' el i32 // !\($'"[]"')

## slice_length

In [ ]:
inl slice_length forall t dim. (x : slice' t dim) : unativeint =
    !\($'"!x.len()"')

## slice_range

In [ ]:
inl slice_range forall t dim. (start : range t) (end : range t) (s : slice' t dim) : rust.ref' (slice' t dim) =
    inl len = s |> slice_length
    inl start, (end : unativeint) =
        match start, end with
        | Start start, End fn => start, len |> convert |> fn |> convert
        | End start_fn, End end_fn => len |> convert |> start_fn, len |> convert |> end_fn |> convert
    match start, end with
    | start, end when unbox end =. len => !\($'"&!s[!start..]"')
    | start, end => !\\((start, end), $'"&!s[$0..$1]"')

## new_slice

In [ ]:
inl new_slice forall el dim. (el : el) : slice' el dim =
    !\\(el, $'"[$0; @dim]"')

## as_slice

In [ ]:
inl as_slice forall t. (x : array_base t) : rust.ref' (slice t) =
    inl x = x |> to_vec
    !\($'"!x.as_slice()"')

## slice_to_vec

In [ ]:
inl slice_to_vec forall t. (slice : rust.ref' (slice t)) : vec t =
    !\\(slice, $'"$0.iter().map(|x| *x).collect::<Vec<_>>()"')

## any

In [ ]:
inl any forall t. (fn : t -> bool) (source : array_base t) : bool =
    !\($'"!source.any(|x| !fn(x))"')

## iter_collect vec

In [ ]:
instance iter_collect vec = fun (iter : into_iterator u) =>
    !\($'"!iter.collect::<Vec<_>>()"')

instance iter_collect'' vec = fun (iter : into_iterator (t (u v))) =>
    !\($'"!iter.collect::<Vec<_>>()"')

## new_vec

In [ ]:
inl new_vec forall t. (items : list t) : vec t =
    inl items =
        (items, ("", 0i32))
        ||> listm.foldBack fun (x : t) (acc, i) =>
            inl x = join x
            $'"!x"' +. (if i = 0 then "" else ", ") +. acc, i + 1
        |> fst
    !\($'"vec\![" + !items + "]"')

In [ ]:
//// test
///! rust

types ()
sm'.types ()
[ 0i32; 1 ]
|> new_vec
|> sm'.format_debug'
|> sm'.from_std_string
|> _assert_eq "[0, 1]"

assert_eq / actual: "[0, 1]" / expected: "[0, 1]"
